In [3]:
import numpy as np
import pandas as pd
import os
WORKING_PATH = "C:/Users/carlo/Projects/Astronomy Research/"

In [ ]:
METADATA = ["Time", "Model"]
PHYSICAL_PARAMETERS = np.loadtxt(os.path.join(WORKING_PATH, "Main Scripts 11.9.24/utils/physical_parameters.txt"), dtype=str, delimiter=" ").tolist()
TOTAL_SPECIES = np.loadtxt(os.path.join(WORKING_PATH, "Main Scripts 11.9.24/utils/species.txt"), dtype=str, delimiter=" ").tolist()

In [ ]:
x = np.array([1,2,3,4,5])

print(x/x)
print(10**x)
print(np.log10(10**x))

In [ ]:
df = pd.read_hdf(os.path.join(WORKING_PATH, "Datasets/training.h5"), "autoencoder", start=0, stop=1000)
print(df.head())

In [ ]:
df.reset_index().loc[20].sort_values(ascending=False)[6:]

In [ ]:
import pandas as pd

data = {'A': range(1, 11), 'B': range(11, 21)}
df = pd.DataFrame(data)

df['pair_group'] = df.index // 2

grouped = df.groupby('pair_group')

pair_means = grouped.mean()

for name, group in grouped:
    print(f"Pair {name}:\n", group, "\n")


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

df = pd.read_hdf(os.path.join(WORKING_PATH, "Datasets/training.h5"), "emulator", start=0, stop=5)

df = np.log10(df)

global_min = df.min().min()  # minimum value across all columns
global_max = df.max().max()  # maximum value across all columns

#scaled_df = (df - global_min) / (global_max - global_min)


scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(
    np.array([global_min, global_max]).reshape(-1, 1)
)
print(scaler.data_min_)



scaled_df

In [ ]:
components = [f"Component_{num}" for num in range(1, 12)]
components

In [2]:
"""

INPUT: training.h5, validation.h5, abundances.scalers
OUTPUT: autoencoder.pth
"""

import os
import numpy as np
import pandas as pd
from datetime import datetime
from joblib import load
import torch
from torch import nn, optim, multiprocessing as mp
from torch.cuda.amp import GradScaler, autocast
from torch.distributed import init_process_group, destroy_process_group
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler

### Configurations
WORKING_PATH = "C:/Users/carlo/Projects/Astronomy Research/"
HP = {
    "encoded_dimensions": 11,
    "learning_rate": 0.001,
    "weight_decay": 0,
    "batch_size": 2*4096,
    "shuffle": True,
    "gradient_clipping": 100,
    "early_stopping_tolerance": 7,
    "max_epochs": 999999,
    "hidden_layer": 5
}

METADATA = ["Time", "Model"] 
PHYSICAL_PARAMETERS = np.loadtxt(os.path.join(WORKING_PATH, "Main Scripts 11.9.24/utils/physical_parameters.txt"), dtype=str, delimiter=" ").tolist()
TOTAL_SPECIES = np.loadtxt(os.path.join(WORKING_PATH, "Main Scripts 11.9.24/utils/species.txt"), dtype=str, delimiter=" ").tolist()


### Data Processing Functions
def load_datasets(path):
    training_dataset_path = os.path.join(path, "Datasets/training.h5")
    validation_dataset_path = os.path.join(path, "Datasets/validation.h5")

    training_dataset = pd.read_hdf(training_dataset_path, "autoencoder", start=0, stop=1000).astype(np.float32)
    validation_dataset = pd.read_hdf(validation_dataset_path, "autoencoder", start=0, stop=1000).astype(np.float32)

    training_dataset = training_dataset[TOTAL_SPECIES]
    validation_dataset = validation_dataset[TOTAL_SPECIES]

    return training_dataset, validation_dataset


def autoencoder_preprocessing(abundances_features):
    ### Preprocesses the data for the autoencoder training. Returns a dataloader.
    print("Starting Encoder Preprocessing.")

    # Created using only the training data.
    scalers = load(os.path.join(WORKING_PATH, "Datasets/scalers.plk"))
    abundances_min, abundances_max = scalers["total_species"]

    # Log10 Scale Abundances and then MinMax scale.
    abundances_features = np.log10(abundances_features, dtype=np.float32)

    # Minmax scale all the abundances_features.
    abundances_features = (abundances_features - abundances_min) / (abundances_max - abundances_min)


    return abundances_features

In [ ]:
WORKING_PATH = "C:/Users/carlo/Projects/Astronomy Research/"

training_dataset, validation_dataset = load_datasets(WORKING_PATH)

training = autoencoder_preprocessing(training_dataset)
training_dataset
row = training_dataset.loc[1]

# Sort the values in decreasing order, placing NaN first
sorted_row = row.sort_values(ascending=False, na_position='first')

sorted_row

In [ ]:
import os
from joblib import load
WORKING_PATH = "C:/Users/carlo/Projects/Astronomy Research/"

scalers = load(os.path.join(WORKING_PATH, "Datasets/scalers.plk"))

scalers["encoded_components"] = (
    0,
    1
)

print(scalers)

In [ ]:
WORKING_PATH = "C:/Users/carlo/Projects/Astronomy Research/"
training_dataset_path = os.path.join(WORKING_PATH, "Datasets/training.h5")
training_dataset = pd.read_hdf(training_dataset_path, "autoencoder", start=0).astype(np.float32)

z = training_dataset.mean()

In [ ]:
import numpy as np
import pandas as pd

rows, cols = 100000, 340
data = np.random.rand(rows, cols).astype(np.float32)
df = pd.DataFrame(data, dtype=np.float32)
data1 = df.to_numpy()

# Check if they are now equal
print(np.array_equal(data, data1))


In [39]:
import pandas as pd
import numpy as np
from math import log10, floor

def round_sig(x, sig=3):
    return round(x, sig-int(floor(log10(abs(x))))-1)


# data = {
#     'A': [0.02345253401234, 123453.456, 12.34523e-15, 0.987654, 0.988454],
#     'B': [0.987654, 1.234252452e-4, 2e-15, 0.987654, 0.988454],
# }
# df = pd.DataFrame(data, dtype=np.float32)

df_rounded1 = df.map(lambda x: round_sig(x, 3))
df_rounded1 = df_rounded1.astype(np.float32)
#mask = df_rounded.duplicated(keep="first")
#df[~mask]
#df

#2.15

In [ ]:
import pandas as pd
import numpy as np
from math import log10, floor
from numba import njit

@njit
def round_sig(x, sig=3):
    if x == 0:
        return 0
    return round(x, sig - int(floor(log10(abs(x)))) - 1)

@njit
def round_array(arr, sig=3):
    result = np.empty_like(arr)
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            result[i, j] = round_sig(arr[i, j], sig)
    return result

# Apply rounding function
rounded_data = round_array(data, sig=3)

# Create a rounded DataFrame
df_rounded2 = pd.DataFrame(rounded_data, dtype=np.float32)
df_rounded2 = df_rounded2.astype(np.float32)
print(df_rounded1.equals(df_rounded2))
print(df_rounded1)
print(df_rounded2)

In [49]:
x = 1.3146541*10**-5

test = np.array([[x]])

print(round_array(test, 3))

[[1.31e-05]]


In [30]:
import torch

# Generate large DataFrame on GPU
rows, cols = 100000, 340
data_gpu = torch.rand((rows, cols), dtype=torch.float32, device='cuda')

def round_sig_torch(x, sig=3):
    """Rounds a tensor to a specified number of significant figures."""
    log_abs = torch.log10(torch.abs(x))
    scale = sig - torch.floor(log_abs) - 1
    factor = torch.pow(10, scale)
    rounded = torch.round(x * factor) / factor
    return rounded

# Apply rounding
rounded_data_gpu = round_sig_torch(data_gpu, sig=3)

# Convert back to pandas DataFrame on CPU (optional)
rounded_data_cpu = rounded_data_gpu.cpu().numpy()
df_rounded = pd.DataFrame(rounded_data_cpu, dtype=np.float32)
